# Import necessary depencencies

In [1]:
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
from contractions import CONTRACTION_MAP

import nltk
nltk.download('stopwords')
nltk.download('sentiwordnet')
nltk.download('wordnet')
nltk.download('vader_lexicon')

nlp = spacy.load('en', parse = False, tag=False, entity=False)
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

import text_normalizer as tn


np.set_printoptions(precision=2, linewidth=80)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


# Load and normalize data

In [2]:
!pip install afinn

     |████████████████████████████████| 61kB 2.4MB/s 
  Created wheel for afinn: filename=afinn-0.1-cp36-none-any.whl size=53451 sha256=6a805c80f2aba07fa8e903d168cbd068f16b6cb0cd5e43e068f58a6bf56d8d9d
  Stored in directory: /root/.cache/pip/wheels/b5/1c/de/428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn


In [3]:
df_1 = pd.read_csv(r'/content/sentiment.csv', error_bad_lines=False, sep='\t')
df_2 = pd.read_csv(r'/content/sentiment_tst.csv', error_bad_lines=False, sep='\t')

dataset = pd.concat([df_1, df_2]).reset_index(drop=True)
dataset.shape

(50000, 3)

In [4]:
reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

# extract data for model evaluation
test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]
sample_review_ids = [7626, 3533, 13010]

# normalize dataset
norm_test_reviews = tn.normalize_corpus(test_reviews)

In [120]:
dataset['sentiment'][7626]

'negative'

# Sentiment Analysis with AFINN

In [5]:
from afinn import Afinn

afn = Afinn(emoticons=True)

## Predict sentiment for sample reviews

In [6]:
import pandas as pd

afinn_wl_url = ('https://raw.githubusercontent.com'
                '/fnielsen/afinn/master/afinn/data/AFINN-111.txt')

afinn_wl_df = pd.read_csv(afinn_wl_url,
                          header=None, # no column names
                          sep='\t',  # tab sepeated
                          names=['term', 'value']) #new column names

seed = 808 # seed for sample so results are stable
afinn_wl_df.sample(10, random_state = seed)

,term,value
1852,regret,-2
1285,indifferent,-2
681,disappoints,-2
770,doubts,-1
1644,outmaneuvered,-2
55,admit,-1
1133,haha,3
1160,haunt,-1
2435,wishing,1
21,abused,-3


In [7]:
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list
    ## print list 
    #for x in unique_list: 
        #print (x) 
list1 = unique(norm_test_reviews[7626].split())

In [8]:
# for i in range(len(list1)):
#   print (list1[i])

In [9]:
for review in range(0, len(list1)):
  if afinn_wl_df['term'].str.contains(list1[review]).any():
      print (afinn_wl_df[afinn_wl_df['term'] == list1[review]]['term'], afinn_wl_df[afinn_wl_df['term'] == list1[review]]['value'])
  else:
    continue


Series([], Name: term, dtype: object) Series([], Name: value, dtype: int64)
Series([], Name: term, dtype: object) Series([], Name: value, dtype: int64)
206    avoid
Name: term, dtype: object 206   -1
Name: value, dtype: int64
Series([], Name: term, dtype: object) Series([], Name: value, dtype: int64)
2433    wish
Name: term, dtype: object 2433    1
Name: value, dtype: int64
Series([], Name: term, dtype: object) Series([], Name: value, dtype: int64)
1060    funny
Name: term, dtype: object 1060    4
Name: value, dtype: int64
Series([], Name: term, dtype: object) Series([], Name: value, dtype: int64)
1437    like
Name: term, dtype: object 1437    2
Name: value, dtype: int64
Series([], Name: term, dtype: object) Series([], Name: value, dtype: int64)
1473    lovely
Name: term, dtype: object 1473    3
Name: value, dtype: int64
Series([], Name: term, dtype: object) Series([], Name: value, dtype: int64)
1957    save
Name: term, dtype: object 1957    2
Name: value, dtype: int64
Series([], Name:

In [10]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    print('Predicted Sentiment polarity:', afn.score(review))
    print('-'*60)

REVIEW: I seem to remember a lot of hype about this movie when it came out, but had avoided seeing it throughout the years. I wish I'd waited longer. Maybe this movie was funny in 1988, I don't know. I was younger then, but it didn't seem like the world was that different. Michelle Pfeiffer, lovely as she is, is never convincing. Mercedes Ruehl not only chews scenery, but stuffs it in her cheeks like a gerbil to save for later. Dean Stockwell is about as convincing as a mob boss as James Gandolfini would be as principal dancer for the Bolshoi. And Matthew Modine demonstrated the most pronounced case of delayed puberty I've ever seen. All in all, it's not bad enough to make you want to pluck out your eyes with a melon-baller, but it's not far off.
Actual Sentiment: negative
Predicted Sentiment polarity: 10.0
------------------------------------------------------------
REVIEW: I read Angels and Demons about 3 years ago, and I can honestly say to is one of the few books that I couldn't pu

## Predict sentiment for test dataset

In [11]:
sentiment_polarity = [afn.score(review) for review in test_reviews]
predicted_sentiments = ['positive' if score >= 1.0 else 'negative' for score in sentiment_polarity]

## Evaluate model performance

In [12]:
from sklearn.metrics import roc_curve, auc 

from sklearn import metrics

In [13]:
# Run the metrics function to display the performance of predictive modeling
def display_confusion_matrix(true_labels, predicted_labels, classes=[1,0]):
    
    total_classes = len(classes)
    level_labels = [total_classes*[0], list(range(total_classes))]

    cm = metrics.confusion_matrix(y_true=true_labels, y_pred=predicted_labels, 
                                  labels=classes)
    cm_frame = pd.DataFrame(data=cm, 
                            columns=pd.MultiIndex(levels=[['Predicted:'], classes], 
                                                  codes=level_labels), 
                            index=pd.MultiIndex(levels=[['Actual:'], classes], 
                                                codes=level_labels)) 
    print(cm_frame)

In [14]:
# Run the metrics function to display the performance of predictive modeling 
def get_metrics(true_labels, predicted_labels):
    
    print('Accuracy:', np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        4))
    print('Precision:', np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('Recall:', np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('F1 Score:', np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))

In [15]:
# Run the metrics function to display the performance of predictive modeling 
def display_classification_report(true_labels, predicted_labels, classes=[1,0]):

    report = metrics.classification_report(y_true=true_labels, 
                                           y_pred=predicted_labels, 
                                           labels=classes) 
    print(report)

In [16]:
# Run the metrics function to display the performance of predictive modeling 
def display_model_performance_metrics(true_labels, predicted_labels, classes=[1,0]):
    print('Model Performance metrics:')
    print('-'*30)
    get_metrics(true_labels=true_labels, predicted_labels=predicted_labels)
    print('\nModel Classification report:')
    print('-'*30)
    display_classification_report(true_labels=true_labels, predicted_labels=predicted_labels, 
                                  classes=classes)
    print('\nPrediction Confusion Matrix:')
    print('-'*30)
    display_confusion_matrix(true_labels=true_labels, predicted_labels=predicted_labels, 
                             classes=classes)

In [17]:
# Performance Metrics for Afinn
# Please Note : the module meu is not been provided. 
display_model_performance_metrics(true_labels=test_sentiments, 
                                      predicted_labels=predicted_sentiments, 
                                  classes=['positive', 'negative'])



Model Performance metrics:
------------------------------
Accuracy: 0.7121
Precision: 0.7301
Recall: 0.7121
F1 Score: 0.7067

Model Classification report:
------------------------------
              precision    recall  f1-score   support

    positive       0.66      0.85      0.75      7467
    negative       0.79      0.58      0.67      7533

    accuracy                           0.71     15000
   macro avg       0.73      0.71      0.71     15000
weighted avg       0.73      0.71      0.71     15000


Prediction Confusion Matrix:
------------------------------
                 Predicted:         
                   positive negative
Actual: positive       6348     1119
        negative       3199     4334


# Sentiment Analysis with SentiWordNet

In [18]:
from nltk.corpus import sentiwordnet as swn

awesome = list(swn.senti_synsets('awesome', 'a'))[0]
print('Positive Polarity Score:', awesome.pos_score())
print('Negative Polarity Score:', awesome.neg_score())
print('Objective Score:', awesome.obj_score())

Positive Polarity Score: 0.875
Negative Polarity Score: 0.125
Objective Score: 0.0


## Build model

In [19]:
def analyze_sentiment_sentiwordnet_lexicon(review,
                                           verbose=False):

    # tokenize and POS tag text tokens
    tagged_text = [(token.text, token.tag_) for token in tn.nlp(review)]
    pos_score = neg_score = token_count = obj_score = 0
    # get wordnet synsets based on POS tags
    # get sentiment scores if synsets are found
    for word, tag in tagged_text:
        ss_set = None
        if 'NN' in tag and list(swn.senti_synsets(word, 'n')):
            ss_set = list(swn.senti_synsets(word, 'n'))[0]
        elif 'VB' in tag and list(swn.senti_synsets(word, 'v')):
            ss_set = list(swn.senti_synsets(word, 'v'))[0]
        elif 'JJ' in tag and list(swn.senti_synsets(word, 'a')):
            ss_set = list(swn.senti_synsets(word, 'a'))[0]
        elif 'RB' in tag and list(swn.senti_synsets(word, 'r')):
            ss_set = list(swn.senti_synsets(word, 'r'))[0]
        # if senti-synset is found        
        if ss_set:
            # add scores for all found synsets
            pos_score += ss_set.pos_score()
            neg_score += ss_set.neg_score()
            obj_score += ss_set.obj_score()
            token_count += 1
    
    # aggregate final scores
    final_score = pos_score - neg_score
    norm_final_score = round(float(final_score) / token_count, 2)
    final_sentiment = 'positive' if norm_final_score >= 0 else 'negative'
    if verbose:
        norm_obj_score = round(float(obj_score) / token_count, 2)
        norm_pos_score = round(float(pos_score) / token_count, 2)
        norm_neg_score = round(float(neg_score) / token_count, 2)
        # to display results in a nice table
        sentiment_frame = pd.DataFrame([[final_sentiment, norm_obj_score, norm_pos_score, 
                                         norm_neg_score, norm_final_score]],
                                       columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                             ['Predicted Sentiment', 'Objectivity',
                                                              'Positive', 'Negative', 'Overall']], 
                                                             codes=[[0,0,0,0,0],[0,1,2,3,4]]))
        print(sentiment_frame)
        
    return final_sentiment

## Predict sentiment for sample reviews

In [20]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    pred = analyze_sentiment_sentiwordnet_lexicon(review, verbose=True)    
    print('-'*60)

REVIEW: I seem to remember a lot of hype about this movie when it came out, but had avoided seeing it throughout the years. I wish I'd waited longer. Maybe this movie was funny in 1988, I don't know. I was younger then, but it didn't seem like the world was that different. Michelle Pfeiffer, lovely as she is, is never convincing. Mercedes Ruehl not only chews scenery, but stuffs it in her cheeks like a gerbil to save for later. Dean Stockwell is about as convincing as a mob boss as James Gandolfini would be as principal dancer for the Bolshoi. And Matthew Modine demonstrated the most pronounced case of delayed puberty I've ever seen. All in all, it's not bad enough to make you want to pluck out your eyes with a melon-baller, but it's not far off.
Actual Sentiment: negative
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            positive        0.79     0.11     0.09    0.02
------------------------------------

## Predict sentiment for test dataset

In [21]:
predicted_sentiments = [analyze_sentiment_sentiwordnet_lexicon(review, verbose=False) for review in norm_test_reviews]

## Evaluate model performance

In [22]:
# Performance Metrics on SentiWordNet
# Please Note : the module meu is not been provided.
display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=predicted_sentiments, 
                                  classes=['positive', 'negative'])

Model Performance metrics:
------------------------------
Accuracy: 0.6889
Precision: 0.6921
Recall: 0.6889
F1 Score: 0.6878

Model Classification report:
------------------------------
              precision    recall  f1-score   support

    positive       0.67      0.75      0.71      7467
    negative       0.72      0.63      0.67      7533

    accuracy                           0.69     15000
   macro avg       0.69      0.69      0.69     15000
weighted avg       0.69      0.69      0.69     15000


Prediction Confusion Matrix:
------------------------------
                 Predicted:         
                   positive negative
Actual: positive       5600     1867
        negative       2799     4734


# Sentiment Analysis with VADER

In [23]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Build model

In [24]:
def analyze_sentiment_vader_lexicon(review, 
                                    threshold=0.1,
                                    verbose=False):
    # pre-process text
    review = tn.strip_html_tags(review)
    review = tn.remove_accented_chars(review)
    review = tn.expand_contractions(review)
    
    # analyze the sentiment for review
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    # get aggregate scores and final sentiment
    agg_score = scores['compound']
    final_sentiment = 'positive' if agg_score >= threshold\
                                   else 'negative'
    if verbose:
        # display detailed sentiment statistics
        positive = str(round(scores['pos'], 2)*100)+'%'
        final = round(agg_score, 2)
        negative = str(round(scores['neg'], 2)*100)+'%'
        neutral = str(round(scores['neu'], 2)*100)+'%'
        sentiment_frame = pd.DataFrame([[final_sentiment, final, positive,
                                        negative, neutral]],
                                        columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                                      ['Predicted Sentiment', 'Polarity Score',
                                                                       'Positive', 'Negative', 'Neutral']], 
                                                              codes=[[0,0,0,0,0],[0,1,2,3,4]]))
        print(sentiment_frame)
    
    return final_sentiment

## Predict sentiment for sample reviews

In [25]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    pred = analyze_sentiment_vader_lexicon(review, threshold=0.4, verbose=True)    
    print('-'*60)

REVIEW: I seem to remember a lot of hype about this movie when it came out, but had avoided seeing it throughout the years. I wish I'd waited longer. Maybe this movie was funny in 1988, I don't know. I was younger then, but it didn't seem like the world was that different. Michelle Pfeiffer, lovely as she is, is never convincing. Mercedes Ruehl not only chews scenery, but stuffs it in her cheeks like a gerbil to save for later. Dean Stockwell is about as convincing as a mob boss as James Gandolfini would be as principal dancer for the Bolshoi. And Matthew Modine demonstrated the most pronounced case of delayed puberty I've ever seen. All in all, it's not bad enough to make you want to pluck out your eyes with a melon-baller, but it's not far off.
Actual Sentiment: negative
     SENTIMENT STATS:                                                     
  Predicted Sentiment Polarity Score             Positive Negative Neutral
0            positive           0.82  14.000000000000002%    10.0%

## Predict sentiment for test dataset

In [28]:
predicted_sentiments = [analyze_sentiment_vader_lexicon(review, threshold=0.7, verbose=False) for review in test_reviews]

## Evaluate model performance

In [29]:
# Performance Metrics on VADER
# Please Note : the module meu is not been provided.
display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=predicted_sentiments, 
                                  classes=['positive', 'negative'])



Model Performance metrics:
------------------------------
Accuracy: 0.7217
Precision: 0.7243
Recall: 0.7217
F1 Score: 0.721

Model Classification report:
------------------------------
              precision    recall  f1-score   support

    positive       0.70      0.77      0.73      7467
    negative       0.75      0.67      0.71      7533

    accuracy                           0.72     15000
   macro avg       0.72      0.72      0.72     15000
weighted avg       0.72      0.72      0.72     15000


Prediction Confusion Matrix:
------------------------------
                 Predicted:         
                   positive negative
Actual: positive       5773     1694
        negative       2481     5052


In [ ]:
Model Performance metrics:
------------------------------
Accuracy: 0.7246
Precision: 0.7247
Recall: 0.7246
F1 Score: 0.7246

Model Classification report:
------------------------------
              precision    recall  f1-score   support

    positive       0.72      0.73      0.73      7467
    negative       0.73      0.72      0.72      7533

    accuracy                           0.72     15000
   macro avg       0.72      0.72      0.72     15000
weighted avg       0.72      0.72      0.72     15000

In [2]:
!pip install stanza

     |████████████████████████████████| 235kB 5.9MB/s 
